In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from efficientnet.tfkeras import EfficientNetB0
import math
import random
import cv2
import skimage.io
import glob
import os
import functools
from albumentations import *
from sklearn import model_selection, metrics
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm


from config import Config
from manager import DataManager
from util import get_optimizer
from generator import get_dataset
from model import NeuralNet, fit, predict


#tf.config.set_visible_devices([], 'GPU')
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

from tensorflow.keras.mixed_precision import experimental as mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

1 Physical GPUs, 1 Logical GPUs
Compute dtype: float16
Variable dtype: float32


In [2]:
data = DataManager.get_train_file()

# dataset = get_dataset(
#     dataframe=data, 
#     input_path=Config.input.path,
#     batch_size=Config.train.batch_size,
#     training=True,
#     augment=True,
#     buffer_size=1,
#     cache=False,
# )

# for x,y in dataset.take(1):
#     x = x.numpy()
#     print("x shape =", x.shape)

# fig, axes = plt.subplots(1, Config.train.batch_size, figsize=(20, 20))

# if Config.train.batch_size > 1:
#     for i, ax in enumerate(axes.reshape(-1)):
#         ax.imshow(x[i])
#         ax.axis('off')
# else:
#     axes.imshow(x[0])
#     axes.axis('off')

In [3]:

input_shape = Config.input.input_shape
input_path = Config.input.path
seed = Config.train.seed
folds = Config.train.folds
batch_size = Config.train.batch_size
epochs = Config.train.epochs
accum_steps = Config.train.accum_steps

lr_steps_per_epoch=math.ceil((10616 * (1-1/Config.train.folds)) / Config.train.batch_size)
lr_max=Config.train.learning_rate.max
lr_min=Config.train.learning_rate.min
lr_decay_epochs=Config.train.learning_rate.decay_epochs
lr_warmup_epochs=Config.train.learning_rate.warmup_epochs
lr_power=Config.train.learning_rate.power

units=Config.model.units
dropout=Config.model.dropout
activation=Config.model.activation

loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer = get_optimizer(
    steps_per_epoch=lr_steps_per_epoch,
    lr_max=lr_max,
    lr_min=lr_min,
    decay_epochs=lr_decay_epochs,
    warmup_epochs=lr_warmup_epochs,
    power=lr_power
)
optimizer = mixed_precision.LossScaleOptimizer(optimizer, loss_scale='dynamic')

classes = np.where(data.data_provider == 'karolinska', 6, 0) + data.isup_grade.values
skf = model_selection.StratifiedKFold(
    folds, shuffle=True, random_state=seed).split(data.image_id, y=data.isup_grade)

for fold_num, (train_idx, valid_idx) in enumerate(skf):

    if fold_num == 0:
        model = NeuralNet(
            engine=EfficientNetB0, 
            input_shape=input_shape, 
            units=units,
            dropout=dropout,
            activation=activation,
            weights='noisy-student')
        model.build([None, *input_shape])
        
        train_dataset = get_dataset(
            dataframe=data.iloc[train_idx], 
            input_path=input_path, 
            batch_size=batch_size,
            training=True,
            augment=True,
            buffer_size=1024,
            cache=False,
        )
        
        valid_dataset = get_dataset(
            dataframe=data.iloc[valid_idx], 
            input_path=input_path, 
            batch_size=batch_size,
            training=False,
            augment=False,
            buffer_size=1,
            cache=True,
        )

        best_score = float('-inf')
        for epoch_num in range(epochs):
            
            fit(model, train_dataset, loss_fn, optimizer, accum_steps)

            preds, trues = predict(model, valid_dataset)

            if Config.infer.tta > 1:
                preds = preds.reshape((-1, Config.infer.tta)).mean(-1)
                trues = trues.reshape((-1, Config.infer.tta)).mean(-1)
                
            preds = np.round(preds, 0)
            trues = np.round(trues, 0)
            score = metrics.cohen_kappa_score(trues, preds, weights='quadratic')
            print("\nscore =", score)
            if score > best_score:
                best_score = score
                model.save_weights(f'output/weights/model-{fold_num}-{epoch_num}.h5')

  0%|          | 0/4246 [00:00<?, ?it/s]

>> fitting with accumulated gradients


  0%|          | 0/4246 [00:00<?, ?it/s]


score = 0.7471641767024998
>> fitting with accumulated gradients


  0%|          | 0/4246 [00:00<?, ?it/s]

...

  0%|          | 0/4246 [00:00<?, ?it/s]


score = 0.7204601385235989
>> fitting with accumulated gradients


LR 0.0002323 : LOSS 0.287511:  37%|███▋      | 1587/4246 [20:25<34:12,  1.30it/s]   
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/alex/.virtualenvs/pca/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-4f5bf3d6cb2a>", line 67, in <module>
    fit(model, train_dataset, loss_fn, optimizer, accum_steps)
  File "/home/alex/Projects/kaggle/prostate-cancer-grade-assessment/src/model.py", line 78, in fit
    if (i+1) % accum_steps == 0:
  File "/home/alex/.virtualenvs/pca/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 767, in __call__
    result = self._call(*args, **kwds)
  File "/home/alex/.virtualenvs/pca/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 801, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/home/alex/.virtualenvs/pca/lib/python3.6/site-packages/tensorflow/python/eager/function.py", line 2811, in __call__
    return graph_function._filtered_call(args, kwargs)  # pylint: di

KeyboardInterrupt: 